<a href="https://colab.research.google.com/github/jjangj/Cyberbulliyng/blob/master/Bertinho.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [32]:
!pip install transformers -qq

In [33]:
!pip install datasets -qq
!pip install "tqdm==4.43.0"

In [34]:
!mkdir reconocimiento_cyberacoso_mujeres

mkdir: cannot create directory ‘reconocimiento_cyberacoso_mujeres’: File exists


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [35]:
import numpy as np
import pandas as pd
import torch
from transformers import BertForSequenceClassification, BertTokenizerFast, Trainer, TrainingArguments
from datasets import load_dataset, Dataset
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from transformers import AutoModel, AutoTokenizer

In [36]:
#Carregamento do BERTimbau
model = BertForSequenceClassification.from_pretrained('neuralmind/bert-base-portuguese-cased') 
tokenizer = BertTokenizerFast.from_pretrained('neuralmind/bert-base-portuguese-cased')

Some weights of the model checkpoint at neuralmind/bert-base-portuguese-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the

In [42]:
#Carregamento do Banco de dados
train_dataset, test_dataset = load_dataset('csv', 
                                           data_files='/content/reconocimiento_cyberacoso_mujeres/train_final_short.csv', 
                                           split=['train[:80%]', 'train[80%:]'],
                                           sep=';')

Using custom data configuration default-d327dd5b795a6835


Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-d327dd5b795a6835/0.0.0/9144e0a4e8435090117cea53e6c7537173ef2304525df4a077c435d8ee7828ff. Subsequent calls will reuse this data.


In [43]:
def tokenize(batch):
    return tokenizer(batch['text'], padding=True, truncation=False)

train_dataset = train_dataset.map(tokenize, batched=True, batch_size=len(train_dataset))
test_dataset = test_dataset.map(tokenize, batched=True, batch_size=len(train_dataset))

In [44]:
# Similar a lo anterior
# Debe escribirse 'labels' y no 'label'
train_dataset.set_format('torch', columns=['input_ids', 'attention_mask', 'label'])
test_dataset.set_format('torch', columns=['input_ids', 'attention_mask', 'label'])


In [45]:
#print("Column names: ", train_dataset.column_names)
#print("Tamanhos:",train_dataset.shape  )

In [46]:
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='binary')
    acc = accuracy_score(labels, preds)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

In [47]:
training_args = TrainingArguments(
    output_dir='./results',
    learning_rate=2e-5,
    num_train_epochs=3,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    #fp16=True,
    warmup_steps=30,
    logging_steps=20,
    weight_decay=0.01,
    #evaluate_during_training=True,
    logging_dir='/content/logs',
    evaluation_strategy='epoch'
)

In [48]:
trainer = Trainer(
    model=model,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
)

In [1]:
trainer.train()

NameError: ignored

In [ ]:
trainer.evaluate()